# Shakespeare next word prediction
- #### An RNN is used to predict the next word based on the previous word(s).
- #### Parts from a Shakespeare poem (Shakespeare’s first sonnet) are used to train the network
### **Network Architecture**
1. The input layer is an embedding layer.
2. The first hidden layer is an LSTM layer.
3. The second hidden layer is a dropout layer.
4. The output layer is a dense layer.

In [1]:
import numpy as np

from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dropout, Dense

In [2]:
with open('Shakespeare_Sonnet_1.txt') as file: 
    text = file.read()

In [3]:
#Read the text file as separate lines of text
with open('Shakespeare_Sonnet_1.txt') as file: 
    text = file.read()
    lines = text.lower().split('\n')

print(lines)   

['from fairest creatures we desire increase,', "that thereby beauty's rose might never die,", 'but as the riper should by time decease,', 'his tender heir might bear his memory:', 'but thou contracted to thine own bright eyes,', "feed'st thy light's flame with self-substantial fuel,", 'making a famine where abundance lies,', 'thy self thy foe, to thy sweet self too cruel:', "thou that art now the world's fresh ornament,", 'and only herald to the gaudy spring,', 'within thine own bud buriest thy content,', "and, tender churl, mak'st waste in niggarding:", '   pity the world, or else this glutton be,', "   to eat the world's due, by the grave and thee.", 'when forty winters shall besiege thy brow,', "and dig deep trenches in thy beauty's field,", "thy youth's proud livery so gazed on now,", 'will be a tattered weed of small worth held:', 'then being asked, where all thy beauty lies,', 'where all the treasure of thy lusty days;', 'to say, within thine own deep sunken eyes,', 'were an all-

# By default, the text_to_word_sequence function:
- Removes all punctuation including tabs and newlines.
- Converts all words to lower case.
- Splits the input string into words using space as the separator.

In [4]:
#Define words, vocabulary size and sequences of words as lines
words = text_to_word_sequence(text)
print(words)

['from', 'fairest', 'creatures', 'we', 'desire', 'increase', 'that', 'thereby', "beauty's", 'rose', 'might', 'never', 'die', 'but', 'as', 'the', 'riper', 'should', 'by', 'time', 'decease', 'his', 'tender', 'heir', 'might', 'bear', 'his', 'memory', 'but', 'thou', 'contracted', 'to', 'thine', 'own', 'bright', 'eyes', "feed'st", 'thy', "light's", 'flame', 'with', 'self', 'substantial', 'fuel', 'making', 'a', 'famine', 'where', 'abundance', 'lies', 'thy', 'self', 'thy', 'foe', 'to', 'thy', 'sweet', 'self', 'too', 'cruel', 'thou', 'that', 'art', 'now', 'the', "world's", 'fresh', 'ornament', 'and', 'only', 'herald', 'to', 'the', 'gaudy', 'spring', 'within', 'thine', 'own', 'bud', 'buriest', 'thy', 'content', 'and', 'tender', 'churl', "mak'st", 'waste', 'in', 'niggarding', 'pity', 'the', 'world', 'or', 'else', 'this', 'glutton', 'be', 'to', 'eat', 'the', "world's", 'due', 'by', 'the', 'grave', 'and', 'thee', 'when', 'forty', 'winters', 'shall', 'besiege', 'thy', 'brow', 'and', 'dig', 'deep', 

# Use a Tokenizer from the keras.preprocessing.text library to convert the input text into indexed tokens.

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(words)
tokens = tokenizer.word_index
print(tokens)

{'thou': 1, 'the': 2, 'thy': 3, 'and': 4, 'to': 5, 'that': 6, 'in': 7, 'of': 8, 'thee': 9, 'self': 10, 'be': 11, 'with': 12, 'for': 13, 'his': 14, 'thine': 15, 'when': 16, 'not': 17, 'but': 18, 'so': 19, 'beauty': 20, 'if': 21, 'which': 22, 'art': 23, 'world': 24, 'it': 25, 'from': 26, "beauty's": 27, 'a': 28, 'then': 29, 'all': 30, 'make': 31, 'is': 32, 'do': 33, 'as': 34, 'by': 35, 'time': 36, 'where': 37, 'or': 38, 'on': 39, 'were': 40, 'she': 41, 'one': 42, 'die': 43, 'sweet': 44, 'now': 45, 'this': 46, 'will': 47, 'see': 48, 'another': 49, 'dost': 50, 'love': 51, 'why': 52, 'doth': 53, 'still': 54, 'ten': 55, 'i': 56, 'should': 57, 'eyes': 58, 'look': 59, 'who': 60, 'age': 61, 'those': 62, 'what': 63, 'times': 64, 'shouldst': 65, 'own': 66, 'fresh': 67, 'shall': 68, 'being': 69, 'treasure': 70, 'an': 71, 'shame': 72, 'how': 73, 'more': 74, 'use': 75, 'fair': 76, 'my': 77, 'glass': 78, 'whose': 79, 'some': 80, 'he': 81, 'her': 82, 'shalt': 83, 'live': 84, 'single': 85, 'lives': 86,

In [6]:
#tokenizer.word_index
print(len(np.unique(words)))
print(len(tokenizer.word_index))

580
580


# Get the Vocabulary Size

In [7]:
vocabulary_size = len(tokenizer.word_index) + 1
print(vocabulary_size)

581


# Build Sequences
Use the texts_to_sequences() method to convert lines into sequences of tokens.

In [8]:
# build subsequences of different sizes, starting from 1 to the size of the sequence. 
sequences = tokenizer.texts_to_sequences(lines)
subsequences = []
for sequence in sequences:
    for i in range(1, len(sequence)):
       subsequence = sequence[:i+1]
       subsequences.append(subsequence)

### Use the pad_sequences from the keras.preprocessing.sequences library to pad the subsequences with zeros so that all sequences are made the same size.
### The padding='pre' parameter tells the function to add padding at the beginning of each sequence.

In [9]:
subsequences = []
for sequence in sequences:
    for i in range(1, len(sequence)):
        subsequence = sequence[:i+1]
        subsequences.append(subsequence)
        
sequence_length = max([len(sequence) for sequence in sequences])
sequences = pad_sequences(subsequences, maxlen=sequence_length, padding='pre')

# Build Input and Output
#### The input to the network is all the words of the sequence except the last one, and the output is the last word of the sequence.
#### Then, convert output into categorical data using the to_categorical() method from the keras.utils module.

In [10]:
x, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocabulary_size)

# Train an LSTM to perform NLP

In [11]:
model = Sequential()
model.add(Embedding(vocabulary_size, 100, input_length=sequence_length - 1))
model.add(LSTM(100))
model.add(Dropout(0.1))
model.add(Dense(units=vocabulary_size, activation='softmax'))

In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.fit(x, y, epochs=500)

Epoch 1/500


37/37 [==============================] - 3s 6ms/step - loss: 6.2976 - accuracy: 0.0136
Epoch 2/500
37/37 [==============================] - 0s 5ms/step - loss: 5.8923 - accuracy: 0.0314
Epoch 3/500
37/37 [==============================] - 0s 6ms/step - loss: 5.7874 - accuracy: 0.0348
Epoch 4/500
37/37 [==============================] - 0s 6ms/step - loss: 5.7486 - accuracy: 0.0306
Epoch 5/500
37/37 [==============================] - 0s 6ms/step - loss: 5.7097 - accuracy: 0.0331
Epoch 6/500
37/37 [==============================] - 0s 6ms/step - loss: 5.6695 - accuracy: 0.0246
Epoch 7/500
37/37 [==============================] - 0s 6ms/step - loss: 5.6067 - accuracy: 0.0365
Epoch 8/500
37/37 [==============================] - 0s 6ms/step - loss: 5.5303 - accuracy: 0.0408
Epoch 9/500
37/37 [==============================] - 0s 6ms/step - loss: 5.4291 - accuracy: 0.0450
Epoch 10/500
37/37 [==============================] - 0s 5ms/step - loss: 5.3243 - accuracy: 0.0374
Epoch 1